In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [ ]:
%%capture
!pip install tensorflow
!pip install keras
!pip install netron
!pip install nltk

!pip3 install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nlp
!pip install rouge-score
#!pip install sacrebleu
!pip install git-python
!pip install sentencepiece
#!pip install torchdata
#!pip install transformers
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# **Parsing data**
I parsed DJANGO dataset and splitted this dataset to 16000 training, 1000 development and 1805 test annotations. Each data point consists of a line of Python code together with a manually created natural language description.

In [ ]:
import pandas as pd

df = pd.read_csv('DJANGO.csv')

df.dropna(inplace = True)

In [ ]:
raw_dataset = datasets.load_dataset('csv', data_files={'train':'DJANGO.csv'},
                                     delimiter=',', quotechar= '"'
                                )
                                

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df['code'].isnull().sum()

0

In [ ]:
df.duplicated

<bound method DataFrame.duplicated of                                                       nl  \
0      from threading import local into default name ...   
1                                import module warnings.   
2      from django.conf import settings into default ...   
3      from django.core import signals into default n...   
4      from django.core.cache.backends.base import In...   
...                                                  ...   
18800                      attrs is an empty dictionary.   
18801  cal the method self.startElement with 2 argume...   
18802                           if contents is not None,   
18803  call the method self.characters with an argume...   
18804  call the method self.endElement with an argume...   

                                                    code  
0                            from threading import local  
1                                        import warnings  
2                     from django . conf import settings  
3    

In [ ]:
df.columns

Index(['nl', 'code'], dtype='object')

In [ ]:
train_val_dataset = raw_dataset['train'].train_test_split(shuffle = True, seed = 200, test_size=0.09598511034)

In [ ]:
train_val_dataset

DatasetDict({
    train: Dataset({
        features: ['nl', 'code'],
        num_rows: 17000
    })
    test: Dataset({
        features: ['nl', 'code'],
        num_rows: 1805
    })
})

In [ ]:
train_val_test = train_val_dataset['train'].train_test_split(shuffle = True, seed = 200, test_size=0.05882352941)

In [ ]:
train_val_test

DatasetDict({
    train: Dataset({
        features: ['nl', 'code'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['nl', 'code'],
        num_rows: 1000
    })
})

In [ ]:
train_dataset = train_val_test['train']
val_dataset = train_val_test['test']

In [ ]:
test_dataset = train_val_dataset["test"]

In [ ]:
test_dataset

Dataset({
    features: ['nl', 'code'],
    num_rows: 1805
})

In [ ]:
train_dataset


Dataset({
    features: ['nl', 'code'],
    num_rows: 16000
})

In [ ]:
val_dataset

Dataset({
    features: ['nl', 'code'],
    num_rows: 1000
})

In [ ]:
from datasets import DatasetDict
train_val_test_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset})

In [ ]:
train_val_test_dataset

DatasetDict({
    train: Dataset({
        features: ['nl', 'code'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['nl', 'code'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['nl', 'code'],
        num_rows: 1805
    })
})

In [ ]:
train_dataset= train_val_test_dataset["train"]
val_dataset = train_val_test_dataset["validation"]
test_dataset = train_val_test_dataset["test"]

In [ ]:
train_dataset.to_csv("train.csv")
val_dataset.to_csv("validation.csv")
test_dataset.to_csv("test.csv")

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

226086

In [ ]:
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel

df = pd.DataFrame(train_dataset[:5])
# del df["id"]
for column, typ in train_dataset.features.items():
      if isinstance(typ, ClassLabel):
          df[column] = df[column].transform(lambda i: typ.names[i])
display(HTML(df.to_html()))

,nl,code
0,"define the method set with 5 arguments, self class instance, key, value, timeout set to DEFAULT_TIMEOUT and version set to None.","def set ( self , key , value , timeout = DEFAULT_TIMEOUT , version = None ) :"
1,"call the function dec with an argument func, return the result.",return dec ( func )
2,"call the method parser.add_argument with 3 arguments: string '--database', default set to DEFAULT_DB_ALIAS,","parser . add_argument ( '--database' , default = DEFAULT_DB_ALIAS , help = 'Nominates a database to print the SQL for. Defaults to the ' '""default"" database.' )"
3,"if boundary is an instance of six.text_type class,","if isinstance ( boundary , six . text_type ) :"
4,"define the method delete with self class instance, key and version set to None as arguments.","def delete ( self , key , version = None ) :"


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [ ]:
show_random_elements(raw_dataset["train"])

,nl,code
0,"if _default is None,",if _default is None :
1,"if TYPE equals to RAW,",if TYPE == RAW :
2,if __name__ equals a string '__main__'.,if __name__ == '__main__' :
3,"call the method sys.stderr.write with an argument string "" as a FastCGI application, you will need to get flup from\n"".","sys . stderr . write ( "" as a FastCGI application, you will need to get flup from\n"" )"
4,"define the function register_serializer with 3 arguments: format, serializer_module and serializers set to None.","def register_serializer ( format , serializer_module , serializers = None ) :"


# **Load Data**

In [ ]:
from datasets import load_dataset


### **DJANGO DATASET**

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("AhmedSSoliman/DJANGO")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/AhmedSSoliman___csv/AhmedSSoliman--DJANGO-900507a2df9e7d02/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 1805
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 1000
    })
})